# Idea
- Get first timestamp
- Resample Dataset to same timewindow
- Merge?

In [104]:
DEVICE_SOURCE = "Zien_device2"

TIME_WINDOW = "30s"

In [105]:
from pathlib import Path

import numpy as np
import pandas as pd
import pickle

In [106]:
sys_data_file_path = Path('/media/<User>/DC/IS_Data_Exploration_and_Feature_Engineering_for_an_IoT_Device_Behavior_Fingerprinting_Dataset/sys_system_calls_Zien_device2/merged_data.npz')
assert sys_data_file_path.exists(), 'Data not found'
vocab_file = Path("/media/<User>/DC/IS_Data_Exploration_and_Feature_Engineering_for_an_IoT_Device_Behavior_Fingerprinting_Dataset/sys_system_calls_Zien_device2/vocabulary.pkl")
assert vocab_file.exists(), f"Vocab file at {vocab_file} does not exist."

In [107]:
import sys
repo_base_path = Path("./../../").resolve()
assert str(repo_base_path).endswith("csg_is"), f"{repo_base_path} is not a valid path to the CSG_IS repository" 

sys.path.append(str(repo_base_path))

from py_dataset import get_all_files_df
from py_dataset import read_in_files
from py_dataset import feature_plotting
from py_dataset import feature_selection
from py_dataset import net_feature_engineering
from py_dataset import entropy_feature_engineering
from py_dataset import sys_func

from py_dataset.classes import DataSource

In [108]:
feature_path = repo_base_path / "data_merg" /"top_260_features.pkl"

with open(feature_path, "rb") as f:
    SELECTED_FEATURES = pickle.load(f)

In [109]:
data_path = Path('/media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/code&data/0_raw_collected_data/')
assert data_path.exists()

df = get_all_files_df.main(data_path)
df.head(1)

txt file found, will drop /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/code&data/0_raw_collected_data/Heqing/device1/3_thetick_4h/collections.txt
txt file found, will drop /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/code&data/0_raw_collected_data/Heqing/device2/1_normal/note.txt
                                           file_path
0  /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/...
RangeIndex(start=0, stop=622, step=1)
csv    530
zip     92
Name: filetype, dtype: int64
                                           file_path filetype  filesize_bytes
0  /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/...      csv           49419
data_source
SYS_data        92
FLS_data        91
KERN_data       91
RES_data        91
block_data      91
network_data    91
entropy_data    75
dtype: int64


,file_path,filetype,filesize_bytes,researcher_name,device_name,label,data_source,file_name,device
0,/media//DC/MAP_CreationOfNewDatasetsForDFL/...,csv,49419,Heqing,device1,1_normal,block_data,e4_5f_01_68_35_e6.csv,Heqing_device1


In [110]:
single_dev = df[df["device"] == DEVICE_SOURCE]
assert len(single_dev["device"].unique()) == 1
print(single_dev.shape)

df = single_dev

(58, 9)


### Get First TimeStamp

In [111]:
def prepare_files(row, resample_time_window):
    if row["filetype"] != "csv" and row["filetype"] != "zip":
        return None

    min_t, max_t, df_t = read_in_files.get_all_timestamps(row['file_path'], row["filetype"],
                                            resample_time_window=resample_time_window)

    row["min_t"] = min_t
    row["max_t"] = max_t

    return row

rows = []
for k, row in df.iterrows():
    x = prepare_files(row, resample_time_window=None)

    rows.append(x)

df_timestamps = pd.DataFrame(rows)

In [112]:
START = df_timestamps["min_t"].min()
f'Timestamps between: {df_timestamps["min_t"].min()} and {df_timestamps["max_t"].max()}'

'Timestamps between: 2023-11-15 10:47:15 and 2023-11-18 04:25:51'

### Resample files

In [113]:
def only_return_selected_columns(df: pd.DataFrame) -> pd.DataFrame:
    selected_cols = SELECTED_FEATURES & set(df.columns)
    return df[list(selected_cols)]

def create_window_sliced_depending_on_type(df: pd.DataFrame, start_timestamp: pd.Timestamp) -> pd.DataFrame:
    data_type = DataSource.from_columns(df.columns.tolist())
    print(f"Data type: {data_type}")

    match data_type:
        case (
            DataSource.FileSystem | DataSource.KernelEvents | DataSource.ResourceUsageEvents | DataSource.ResourceUsageEvents_RP4  | DataSource.BlockInputOutputEvents
        ):
            df = only_return_selected_columns(df)
            df = feature_plotting.test_transform_all_to_numeric_columns(df.copy())
            return df.resample(TIME_WINDOW, origin=start_timestamp).mean()
        case DataSource.NetRecords:
            mean_df = net_feature_engineering.create_mean_net_df_single_label(df, time_window=TIME_WINDOW, start=start_timestamp)
            return only_return_selected_columns(mean_df)
        case DataSource.Entropy:
            if "entropy_file_count" not in SELECTED_FEATURES:
                return None
            return entropy_feature_engineering.resample_high_entropy_grouping_by_label_single_label(df, time_window=TIME_WINDOW, start=start_timestamp)
            
        case DataSource.SystemCalls:
            raise NotImplementedError("SystemCalls not implemented yet")
        

def check_if_keep_source(columns):
    data_type = DataSource.from_columns(columns.tolist())
    if data_type in [DataSource.SystemCalls, DataSource.Entropy, DataSource.NetRecords]:
        return True

    chosen_cols_from_source = data_type.value & SELECTED_FEATURES

    return any(chosen_cols_from_source)

In [114]:

df_mean_collection: list[pd.DataFrame] = []

for source, group_df in df.groupby(["data_source"]):
    dfs_same_source = []

    checked_source = False
    for k, row in group_df.iterrows():
        if row["filetype"] != "csv":
            print(f"Skipping {row['file_path']}")
            continue


        df_t = read_in_files.get_all_columns(row['file_path'], row["filetype"])
        if not checked_source:
            if not check_if_keep_source(df_t.columns):
                print(f"Skipping {source}")
                break
            checked_source = True

        try:
            df_mean = create_window_sliced_depending_on_type(df_t, start_timestamp=START)
        except Exception as ex:
            print(f"Problem in file {row['file_path']}")
            raise ex
        
        if df_mean is None:
            print(f"Skipping {row['file_path']}")
            continue
        
        df_mean = df_mean.copy()
        print(f"Shape: {df_mean.shape}")
        df_mean["label"] = row["label"]
        df_mean["device"] = row["device"]
        df_mean["data_source"] = row["data_source"]

        # print(f"Start and End of Index Timestamp: {df_mean.index[0]} - {df_mean.index[-1]}")
        dfs_same_source.append(df_mean)

    if len(dfs_same_source) == 0:
        print(f"Skipping {source}")
        continue
    same_df = pd.concat(dfs_same_source, axis=0)
    print(f"same_df shape: {same_df.shape}")
    df_mean_collection.append(same_df)
    # break

len(df_mean_collection)

/tmp/ipykernel_94367/1828661645.py:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for source, group_df in df.groupby(["data_source"]):


Data type: DataSource.FileSystem
Shape: (480, 54)
Data type: DataSource.FileSystem
Shape: (481, 54)
Data type: DataSource.FileSystem
Shape: (481, 54)
Data type: DataSource.FileSystem
Shape: (480, 54)
Data type: DataSource.FileSystem
Shape: (481, 54)
Data type: DataSource.FileSystem
Shape: (481, 54)
Data type: DataSource.FileSystem
Shape: (481, 54)
Data type: DataSource.FileSystem
Shape: (481, 54)
Data type: DataSource.FileSystem
Shape: (481, 54)
same_df shape: (4327, 57)
Data type: DataSource.KernelEvents
Shape: (481, 56)
Data type: DataSource.KernelEvents
Shape: (481, 56)
Data type: DataSource.KernelEvents
Shape: (481, 56)
Data type: DataSource.KernelEvents
Shape: (481, 56)
Data type: DataSource.KernelEvents
Shape: (482, 56)
Data type: DataSource.KernelEvents
Shape: (481, 56)
Data type: DataSource.KernelEvents
Shape: (481, 56)
Data type: DataSource.KernelEvents
Shape: (481, 56)
Data type: DataSource.KernelEvents
Shape: (481, 56)
same_df shape: (4330, 59)
Data type: DataSource.Resource

5

#### Remove Dublicate Columns and Concatenate

=> some dont work => if Index error use merge manually

In [115]:
try:
    pd.concat(df_mean_collection, axis=1)
except Exception as ex:
    print(ex)

Reindexing only valid with uniquely valued Index objects


In [116]:
from collections import defaultdict

cols = defaultdict(int)
for dfs in df_mean_collection:
    for k in dfs.columns.tolist():
        cols[k] +=1

print("Dublicates:", [k for k, v in cols.items() if v > 1])

to_ignore = ["data_source", "label", "device"]

infos = pd.DataFrame(columns=to_ignore)

for dfs in df_mean_collection:
    for k in dfs.columns.tolist():
        if k in to_ignore:
            continue

        if cols[k] > 1:
            print(k)

            dfs.rename(columns={k: f'{k}_{dfs["data_source"].iloc[0]}'}, inplace=True)

    assert dfs["device"].nunique() == 1
    assert dfs["label"].nunique() == 9
    assert dfs["data_source"].nunique() == 1
    
    infos = pd.concat([infos,dfs[to_ignore]], axis=0)

    # for t,row in dfs.iterrows():
        # infos.append({"device": row["device"], "label": row["label"], "data_source": row["data_source"], "timestamp": t})
    
    dfs.drop(columns=to_ignore, inplace=True)

Dublicates: ['block:block_bio_backmerge', 'writeback:writeback_dirty_inode', 'writeback:writeback_dirty_inode_enqueue', 'writeback:global_dirty_state', 'ext4:ext4_free_inode', 'jbd2:jbd2_checkpoint_stats', 'block:block_dirty_buffer', 'block:block_getrq', 'writeback:writeback_mark_inode_dirty', 'block:block_touch_buffer', 'ext4:ext4_writepages_result', 'writeback:writeback_dirty_page', 'ext4:ext4_unlink_enter', 'ext4:ext4_releasepage', 'ext4:ext4_es_lookup_extent_enter', 'seconds', 'writeback:sb_clear_inode_writeback', 'ext4:ext4_journal_start', 'ext4:ext4_da_update_reserve_space', 'jbd2:jbd2_handle_start', 'filemap:mm_filemap_delete_from_page_cache', 'block:block_unplug', 'jbd2:jbd2_start_commit', 'block:block_bio_remap', 'filemap:mm_filemap_add_to_page_cache', 'jbd2:jbd2_handle_stats', 'label', 'device', 'data_source', 'cpu-migrations', 'task:task_newtask', 'kmem:kmem_cache_alloc', 'kmem:kfree', 'timer:timer_start', 'kmem:mm_page_alloc_zone_locked', 'tcp:tcp_probe', 'sched:sched_proce

In [117]:
new_dfs = []
for adf in df_mean_collection:
    print(adf.shape)
    adf = adf[~adf.index.duplicated(keep='first')]
    new_dfs.append(adf)

(4327, 54)
(4330, 56)
(4329, 73)
(4336, 3)
(4330, 2)


In [118]:
df_all = pd.concat(new_dfs, axis=1)
df_all.sample(2)

,block:block_bio_backmerge_FLS_data,writeback:writeback_dirty_inode_FLS_data,writeback:writeback_dirty_inode_enqueue_FLS_data,ext4:ext4_unlink_exit,ext4:ext4_begin_ordered_truncate,ext4:ext4_ext_remove_space_done,writeback:global_dirty_state_FLS_data,ext4:ext4_free_inode_FLS_data,jbd2:jbd2_checkpoint_stats_FLS_data,jbd2:jbd2_commit_locking,...,napi:napi_poll,tcp:tcp_probe_RES_data,pkt_in,jbd2:jbd2_handle_stats_RES_data,branch-instructions,write_ops,util,write_kbs,(OLD) AverageLength,(OLD) VarianceLength
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-11-15 21:40:45,5.0,1333.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,659.000000,681.000000,1437.666667,6.333333,9.687082e+08,3.333333,10.113333,0.500000,426.865116,371558.472376
2023-11-16 22:59:45,3.0,576.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,668.333333,633.666667,2110.666667,87.333333,7.525382e+08,4.600000,5.730000,0.933333,1071.887863,413159.156497


In [119]:
df_all = df_all.merge(infos["label"], how='outer', left_index=True, right_index=True)

In [120]:
df_all.shape, len(SELECTED_FEATURES)

((21652, 189), 260)

In [121]:
df_all

,block:block_bio_backmerge_FLS_data,writeback:writeback_dirty_inode_FLS_data,writeback:writeback_dirty_inode_enqueue_FLS_data,ext4:ext4_unlink_exit,ext4:ext4_begin_ordered_truncate,ext4:ext4_ext_remove_space_done,writeback:global_dirty_state_FLS_data,ext4:ext4_free_inode_FLS_data,jbd2:jbd2_checkpoint_stats_FLS_data,jbd2:jbd2_commit_locking,...,tcp:tcp_probe_RES_data,pkt_in,jbd2:jbd2_handle_stats_RES_data,branch-instructions,write_ops,util,write_kbs,(OLD) AverageLength,(OLD) VarianceLength,label
2023-11-15 10:47:15,136.5,472.0,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,...,169.000000,492.500000,0.0,6.536332e+08,1.066667,5.333333,0.2,830.782820,480407.287172,1_normal
2023-11-15 10:47:15,136.5,472.0,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,...,169.000000,492.500000,0.0,6.536332e+08,1.066667,5.333333,0.2,830.782820,480407.287172,1_normal
2023-11-15 10:47:15,136.5,472.0,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,...,169.000000,492.500000,0.0,6.536332e+08,1.066667,5.333333,0.2,830.782820,480407.287172,1_normal
2023-11-15 10:47:15,136.5,472.0,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,...,169.000000,492.500000,0.0,6.536332e+08,1.066667,5.333333,0.2,830.782820,480407.287172,1_normal
2023-11-15 10:47:15,136.5,472.0,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,...,169.000000,492.500000,0.0,6.536332e+08,1.066667,5.333333,0.2,830.782820,480407.287172,1_normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-18 04:25:15,0.0,394.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,611.666667,2089.333333,0.0,7.921394e+08,0.000000,0.000000,0.0,858.217039,485063.710929,7_backdoor
2023-11-18 04:25:15,0.0,394.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,611.666667,2089.333333,0.0,7.921394e+08,0.000000,0.000000,0.0,858.217039,485063.710929,7_backdoor
2023-11-18 04:25:15,0.0,394.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,611.666667,2089.333333,0.0,7.921394e+08,0.000000,0.000000,0.0,858.217039,485063.710929,7_backdoor
2023-11-18 04:25:15,0.0,394.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,611.666667,2089.333333,0.0,7.921394e+08,0.000000,0.000000,0.0,858.217039,485063.710929,7_backdoor


# Add Sys Call Data

In [122]:
def read_in_preprocessed_Xi_sensor_2():
    with np.load(sys_data_file_path, allow_pickle=True) as npz:
        print(npz.files)
        X = npz['arr_0']
    print(X.shape)

    vectorizer = sys_func.create_CountVectorizer(vocab_file=vocab_file)
    cols = vectorizer.get_feature_names_out()
    cols = np.append(cols, [['timestamp'], ['label']])
    print (cols.shape)
    df = pd.DataFrame(X, columns=cols)
    # df.set_index('timestamp', inplace=True)
    read_in_files._clean_and_set_index_timestamp(df)

    label_encoder = sys_func.create_label_encoder_for_labels()
    df['label'] = label_encoder.inverse_transform(df['label'].values.astype(int))

    vectors = df.copy()
    print(vectors.shape)
    
    return vectors

sys_df = read_in_preprocessed_Xi_sensor_2()
# sys_df = only_return_selected_columns(sys_df)
# sys_df_mean = sys_df.resample(TIME_WINDOW, origin=START).mean()

['arr_0']
(11183, 158)
Loaded set: ('accept', 'accept4', 'access', 'add_key', 'adjtimex', 'bind', 'brk', 'capget', 'capset', 'chdir', 'chmod', 'chown32', 'clock_gettime', 'clone', 'close', 'connect', 'dup', 'dup2', 'dup3', 'epoll_ctl', 'epoll_wait', 'eventfd2', 'execve', 'exit', 'exit_group', 'faccessat', 'fadvise64_64', 'fallocate', 'fchdir', 'fchmod', 'fchmodat', 'fchown32', 'fcntl64', 'fgetxattr', 'flistxattr', 'flock', 'fsetxattr', 'fstat64', 'fstatat64', 'fstatfs64', 'fsync', 'ftruncate64', 'futex', 'getcwd', 'getdents64', 'getegid32', 'geteuid32', 'getgid32', 'getgroups32', 'getpeername', 'getpgid', 'getpgrp', 'getpid', 'getppid', 'getpriority', 'getrandom', 'getresgid32', 'getresuid32', 'getsid', 'getsockname', 'getsockopt', 'gettid', 'getuid32', 'getxattr', 'inotify_add_watch', 'inotify_rm_watch', 'ioctl', 'ioprio_set', 'kcmp', 'keyctl', 'kill', 'lchown32', 'lgetxattr', 'llseek', 'lstat64', 'madvise', 'mkdir', 'mknod', 'mmap2', 'mount', 'mprotect', 'mremap', 'msync', 'munmap', 

In [123]:
sys_df.head(2)

,accept,accept4,access,add_key,adjtimex,bind,brk,capget,capset,chdir,...,uname,unlink,unlinkat,utimensat,utimes,wait4,waitid,write,writev,label
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-11-15 10:47:15,0.0,0.000185,0.006648,0.0,0.0,0.000128,0.008571,0.000206,0.000048,0.000092,...,0.002107,0.000021,0.0,0.0,0.0,0.002843,0.000103,0.008448,0.000024,1_normal
2023-11-15 10:47:27,0.0,0.000421,0.020220,0.0,0.0,0.000510,0.022782,0.000796,0.000274,0.000052,...,0.007827,0.000095,0.0,0.0,0.0,0.007827,0.000235,0.016586,0.000054,1_normal


In [124]:
if sys_df["label"].unique().shape[0] != 9:
    raise Exception(f'Not all labels!!!: {sys_df["label"].unique()}')
sys_df["label"].unique()

array(['1_normal', '9_xmrig', '4_bashlite', '2_ransomware', '3_thetick',
       '5_httpbackdoor', '6_beurk', '8_bdvl', '7_backdoor'], dtype=object)

In [125]:
sys_df = only_return_selected_columns(sys_df)
sys_df_mean = sys_df.resample(TIME_WINDOW, origin=START).mean()

In [126]:
sys_df_mean.head(2)

,timerfd_settime,setuid32,llseek,getpid,setgroups32,kcmp,sched_yield,rmdir,getpeername,mkdir,...,lstat64,access,mmap2,setitimer,rt_sigprocmask,accept4,chdir,clone,connect,waitid
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-11-15 10:47:15,0.001815,0.000040,0.322643,0.122633,0.000103,0.001990,0.0,0.000411,0.000097,0.000657,...,0.288095,0.014252,0.112456,0.000145,0.025027,0.000351,0.000085,0.003958,0.006158,0.000246
2023-11-15 10:47:45,0.002624,0.000059,0.395714,0.167200,0.000000,0.003783,0.0,0.000686,0.000097,0.000841,...,0.003067,0.019017,0.137930,0.000141,0.030809,0.000491,0.000127,0.005291,0.008841,0.000395


In [127]:
all_df1 = df_all.merge(sys_df_mean, how='outer', left_index=True, right_index=True)
print(f"all_df1 shape: {all_df1.shape}")

all_df1 shape: (25676, 286)


In [128]:
all_df1.head(4)

,block:block_bio_backmerge_FLS_data,writeback:writeback_dirty_inode_FLS_data,writeback:writeback_dirty_inode_enqueue_FLS_data,ext4:ext4_unlink_exit,ext4:ext4_begin_ordered_truncate,ext4:ext4_ext_remove_space_done,writeback:global_dirty_state_FLS_data,ext4:ext4_free_inode_FLS_data,jbd2:jbd2_checkpoint_stats_FLS_data,jbd2:jbd2_commit_locking,...,lstat64,access,mmap2,setitimer,rt_sigprocmask,accept4,chdir,clone,connect,waitid
2023-11-15 10:47:15,136.5,472.0,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,...,0.288095,0.014252,0.112456,0.000145,0.025027,0.000351,0.000085,0.003958,0.006158,0.000246
2023-11-15 10:47:15,136.5,472.0,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,...,0.288095,0.014252,0.112456,0.000145,0.025027,0.000351,0.000085,0.003958,0.006158,0.000246
2023-11-15 10:47:15,136.5,472.0,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,...,0.288095,0.014252,0.112456,0.000145,0.025027,0.000351,0.000085,0.003958,0.006158,0.000246
2023-11-15 10:47:15,136.5,472.0,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,...,0.288095,0.014252,0.112456,0.000145,0.025027,0.000351,0.000085,0.003958,0.006158,0.000246


In [129]:
spath = repo_base_path / "training" / "data" / "all_30s" / f"all_df_{TIME_WINDOW}_{DEVICE_SOURCE}.csv"

all_df1.to_csv(str(spath))